In [6]:
import json
import pydgraph

from pydgraph import DgraphClient, DgraphClientStub
from grapl_analyzerlib.schemas import *

from grapl_analyzerlib.schemas.schema_builder import ManyToMany

In [7]:
class AnyNodeSchema(NodeSchema):
    @staticmethod
    def self_type() -> str:
        return 'Any'

class RiskSchema(NodeSchema):
    def __init__(self):
        super(RiskSchema, self).__init__()
        (
            self
            .with_str_prop('analyzer_name')
            .with_int_prop('risk_score')
        )
    
    @staticmethod
    def self_type() -> str:
        return 'Risk'
    
class LensSchema(NodeSchema):
    def __init__(self):
        super(LensSchema, self).__init__()
        (
            self
            .with_str_prop('lens')
            .with_int_prop('score')
            .with_forward_edge('scope', ManyToMany(AnyNodeSchema), 'in_scope')
        )
    
    @staticmethod
    def self_type() -> str:
        return 'Lens'
    
def set_schema(client, schema, engagement=False):
    op = pydgraph.Operation(schema=schema)
    client.alter(op)
                 
                
def drop_all(client):
    op = pydgraph.Operation(drop_all=True)
    client.alter(op)
    
def format_schemas(schema_defs):
    schemas = "\n\n".join([schema.to_schema_str() for schema in schema_defs])

    types = "\n\n".join([schema.generate_type() for schema in schema_defs])

    return "\n".join([
        "  # Type Definitions",
        types,
        "\n  # Schema Definitions",
        schemas,
    ])

In [8]:
mclient = DgraphClient(DgraphClientStub('alpha0.mastergraphcluster.grapl:9080'))
eclient = DgraphClient(DgraphClientStub('alpha0.engagementgraphcluster.grapl:9080'))

In [10]:
# drop_all(mclient)
# drop_all(eclient)

schemas = (
    ProcessSchema(),
    FileSchema(),
    IpConnectionSchema(),
    IpAddressSchema(),
    IpPortSchema(),
    NetworkConnectionSchema(),
    ProcessInboundNetworkConnectionSchema(),
    ProcessOutboundNetworkConnectionSchema(),  
)

schema_str = format_schemas(schemas)
set_schema(mclient, schema_str)


risk_schema = RiskSchema()
lens_schema = LensSchema()
eg_schemas = [s.with_forward_edge('risks', ManyToMany(RiskSchema), 'risky_nodes') for s in schemas]
eg_schemas.extend([risk_schema, lens_schema])
eg_schema_str = format_schemas(eg_schemas)
set_schema(eclient, eg_schema_str)

In [ ]:
import os
import string

from hashlib import blake2b, sha256
from random import randint, choice

import boto3

def hash_password(cleartext, salt) -> str:
    hashed = sha256(cleartext).digest()
    hasher = blake2b(salt=salt)
    hasher.update(hashed)
    return hasher.digest()


def create_user(username, cleartext):
    assert cleartext
    dynamodb = boto3.resource('dynamodb')
    table = dynamodb.Table('user_auth_table')
    # We hash before calling 'hashed_password' because the frontend will also perform
    # client side hashing
    cleartext += "f1dafbdcab924862a198deaa5b6bae29aef7f2a442f841da975f1c515529d254";

    hashed = sha256(cleartext.encode('utf8')).hexdigest()
    
    for i in range(0, 5000):
        hashed = sha256(hashed.encode('utf8')).hexdigest()
    
    print(hashed)
    salt = os.urandom(blake2b.SALT_SIZE)
    password = hash_password(hashed.encode('utf8'), salt)
#     str(salt)
#     str(password)
    table.put_item(
        Item={
            'username': username,
            'salt': salt,
            'password': password
        }
    )
    
    

allchar = string.ascii_letters + string.punctuation + string.digits
password = "".join(choice(allchar) for x in range(randint(14, 16)))
print(f'your password is {password}')
username = ''
assert username, 'Replace the username with your desired username'
create_user(username, password)
password = ""
print("""Make sure to clear this cell and restart the notebook to ensure your password does not leak!""")

In [ ]:
# CLEAR CACHE
def clear_redis_caches():
    from redis import Redis

    def chunker(seq, size):
        return [seq[pos:pos + size] for pos in range(0, len(seq), size)]
    red = Redis()
    CACHE_ADDRS = [
        "gra-gr-1h7vxdadzip98.b2fixp.0001.use1.cache.amazonaws.com",
        "gra-gr-1o2uc0akc68z4.b2fixp.0001.use1.cache.amazonaws.com",
        "gra-gr-2mhiu3q9eilo.b2fixp.0001.use1.cache.amazonaws.com"
    ]
    CACHE_PORT = 6379
    for CACHE_ADDR in CACHE_ADDRS:
        r = Redis(host=CACHE_ADDR, port=CACHE_PORT, db=0, decode_responses=True)

        for keys in chunker([k for k in r.keys()], 10000):
            r.delete(*keys)

In [52]:
# CLEAR CACHE
def clear_redis_caches():
    from redis import Redis

    def chunker(seq, size):
        return [seq[pos:pos + size] for pos in range(0, len(seq), size)]
    red = Redis()
    CACHE_ADDRS = [
        "gra-gr-1h7vxdadzip98.b2fixp.0001.use1.cache.amazonaws.com",
        "gra-gr-1o2uc0akc68z4.b2fixp.0001.use1.cache.amazonaws.com",
        "gra-gr-2mhiu3q9eilo.b2fixp.0001.use1.cache.amazonaws.com"
    ]
    CACHE_PORT = 6379
    for CACHE_ADDR in CACHE_ADDRS:
        r = Redis(host=CACHE_ADDR, port=CACHE_PORT, db=0, decode_responses=True)

        for keys in chunker([k for k in r.keys()], 10000):
            r.delete(*keys)

In [14]:
import time
import boto3

def reset_db(table_names):

    for table_name in table_names:
        dynamodb = boto3.client('dynamodb', region_name='us-east-1')
        table_description = dynamodb.describe_table(TableName=table_name)

        print(table_description['Table'])

        # delete table

        try:
            dynamodb.delete_table(TableName=table_name)
        except Exception as e:
            print('failed to delete {}'.format(e))
            try:
                dynamodb.delete_table(TableName=table_name)
            except Exception as e:
                print('failed to delete {}'.format(e))

        time.sleep(8)

        try:
            dynamodb.create_table(
                TableName=table_name,
                AttributeDefinitions=table_description['Table']['AttributeDefinitions'],

                BillingMode='PAY_PER_REQUEST',

                KeySchema=table_description['Table']['KeySchema'],

            )
        except:
            time.sleep(5)
            dynamodb.create_table(
                TableName=table_name,
                AttributeDefinitions=table_description['Table']['AttributeDefinitions'],

                BillingMode='PAY_PER_REQUEST',

                KeySchema=table_description['Table']['KeySchema'],

            )
reset_db([
       'process_history_table',
       'file_history_table',
       'node_id_retry_table'
    ])


ClientError: An error occurred (AccessDeniedException) when calling the DescribeTable operation: User: arn:aws:sts::251074890252:assumed-role/engagements-notebook-stac-engagementsnotebookroleB-15FW0GQCXDTU3/SageMaker is not authorized to perform: dynamodb:DescribeTable on resource: arn:aws:dynamodb:us-east-1:251074890252:table/process_history_table